# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
# Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [3]:
# Your code here; use a subquery. No join is necessary 
cur.execute("""SELECT customerNumber,
                      contactLastName,
                      contactFirstName
                      FROM customers
                      WHERE customerNumber IN (SELECT customerNumber 
                                               FROM orders
                                               WHERE orderDate IS '2003-01-31'
                                              );""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [19]:
# Your code here; use a subquery. No join is necessary 
cur.execute("""SELECT productCode,
                      productName,
                      count(orderNumber),
                      sum(quantityOrdered)
                      FROM products
                      JOIN orderdetails USING(productCode)
                      GROUP BY productCode
                      ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df ### How is no join necessary???

,productCode,productName,count(orderNumber),sum(quantityOrdered)
0,S10_1678,1969 Harley Davidson Ultimate Chopper,28,1057
1,S10_1949,1952 Alpine Renault 1300,28,961
2,S10_2016,1996 Moto Guzzi 1100i,28,999
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,28,985
4,S10_4757,1972 Alfa Romeo GTA,28,1030
...,...,...,...,...
104,S700_3505,The Titanic,27,952
105,S700_3962,The Queen Mary,27,896
106,S700_4002,American Airlines: MD-11S,28,1085
107,S72_1253,Boeing X-32A JSF,28,960


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [24]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
# Your code here; use a subquery. No join is necessary 
cur.execute("""SELECT productName,
                      count(DISTINCT customerNumber)
                      FROM products
                      JOIN orderdetails USING(productCode)
                      JOIN orders USING(orderNumber)
                      GROUP BY productName
                      ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productName,count(DISTINCT customerNumber)
0,18th Century Vintage Horse Carriage,26
1,18th century schooner,24
2,1900s Vintage Bi-Plane,26
3,1900s Vintage Tri-Plane,22
4,1903 Ford Model A,21
...,...,...
104,The Mayflower,22
105,The Queen Mary,24
106,The Schooner Bluenose,23
107,The Titanic,22


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [32]:
# Your code here
cur.execute("""SELECT DISTINCT employeeNumber,
                      firstName,
                      lastName,
                      offices.city,
                      officeCode
                      FROM employees
                      JOIN offices USING(officeCode)
                      JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
                      JOIN orders USING(customerNumber)
                      JOIN orderdetails USING(orderNumber) 
                      
                      WHERE productCode IN (SELECT productCode
                                            FROM products
                                            JOIN orderdetails USING(productCode)
                                            JOIN orders USING(orderNumber)
                                            GROUP BY productCode
                                            HAVING COUNT(DISTINCT customerNumber) > 20
                                            );""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,city,officeCode
0,1216,Steve,Patterson,Boston,2
1,1504,Barry,Jones,London,7
2,1286,Foon Yue,Tseng,NYC,3
3,1370,Gerard,Hernandez,Paris,4
4,1401,Pamela,Castillo,Paris,4
5,1323,George,Vanauf,NYC,3
6,1621,Mami,Nishi,Tokyo,5
7,1501,Larry,Bott,London,7
8,1165,Leslie,Jennings,San Francisco,1
9,1337,Loui,Bondur,Paris,4


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [46]:
# Your code here
cur.execute("""SELECT DISTINCT employeeNumber,
                      firstName,
                      lastName,
                      count(customerNumber),
                      AVG(creditLimit)
                      FROM employees
                      JOIN offices USING(officeCode)
                      JOIN customers 
                      ON employees.employeeNumber = customers.salesRepEmployeeNumber
                      GROUP BY employeeNumber
                      
                      HAVING AVG(creditLimit) > 15000
                      ORDER BY AVG(creditLimit)
                      
                      ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,count(customerNumber),AVG(creditLimit)
0,1166,Leslie,Thompson,6,65266.666667
1,1286,Foon Yue,Tseng,7,66614.285714
2,1702,Martin,Gerard,6,71916.666667
3,1188,Julie,Firrelli,6,73916.666667
4,1323,George,Vanauf,8,80887.500000
5,1401,Pamela,Castillo,10,81340.000000
6,1216,Steve,Patterson,6,81533.333333
7,1621,Mami,Nishi,5,83900.000000
8,1504,Barry,Jones,9,85511.111111
9,1611,Andy,Fixter,5,86060.000000


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!